In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

### Validate Endpoint

In [ ]:
new_request = {}
new_request["messages"] = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "What’s in this image?"},
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                },
            },
        ],
    }
]
new_request["temperature"] = 0.5

In [ ]:
from utils import display_messages
display_messages(new_request["messages"])

In [ ]:
from openai_api import send_openai_request
response = send_openai_request(new_request, model="internvlm-76b")
print(response)

In [ ]:
from gemini_api import convert_openai_to_gemini, send_gemini_request
gemini_request = convert_openai_to_gemini(new_request)
send_gemini_request(gemini_request, model="gemini-1.5-flash-latest")
print(response)

### Vision Anomaly Detection

#### View Time Series Data

In [ ]:
from prompt import time_series_to_image
from utils import view_base64_image, display_messages
from data.synthetic import SyntheticDataset

data_name = 'trend'
data_dir = f'data/synthetic/{data_name}/eval/'
train_dir = f'data/synthetic/{data_name}/train/'
eval_dataset = SyntheticDataset(data_dir)
eval_dataset.load()
train_dataset = SyntheticDataset(train_dir)
train_dataset.load()

In [ ]:
from prompt import create_openai_request
from openai_api import send_openai_request
import matplotlib.pyplot as plt

# Set default DPI
plt.rcParams['figure.dpi'] = 240

In [ ]:
for i in range(5):
    data = eval_dataset[i]
    view_base64_image(
        time_series_to_image(
            data[1], 
            gt_anomaly_intervals=data[0],
            fig_size=(20, 3)
        )
    )

In [ ]:
request = create_openai_request(
    eval_dataset[0][1],
    vision=True,
    few_shots=train_dataset.few_shots(num_shots=1)
)
display_messages(request["messages"])

In [ ]:
response = send_openai_request(request, model='gpt-4o')
response

In [ ]:
view_base64_image(
    time_series_to_image(
        eval_dataset[0][1], 
        gt_anomaly_intervals=eval_dataset[0][0], 
        anomalies={'internvlm-76b': response},
        fig_size=(20, 3)
    )
)

In [ ]:
from gemini_api import convert_openai_to_gemini, send_gemini_request
request = create_openai_request(
    eval_dataset[0][1],
    vision=True,
    few_shots=train_dataset.few_shots(num_shots=1)
)
response = send_openai_request(request, model='gemini-1.5-pro-latest')
response

In [ ]:
view_base64_image(
    time_series_to_image(
        eval_dataset[0][1], 
        gt_anomaly_intervals=eval_dataset[0][0], 
        anomalies={'gemini': response},
        fig_size=(20, 3)
    )
)

### Going through a dataset

In [ ]:
from utils import online_AD_with_retries
from prompt import create_openai_request
from openai_api import send_openai_request

In [ ]:
# online_AD_with_retries(
#     model_name='internvlm',
#     data_name='point',
#     ad_func=intern_ad,
#     variant="1shot_vision"
# )

### Generate OpenAI Batch

In [ ]:
from utils import generate_batch_AD_requests
from prompt import create_openai_request

def openai_request(series, train_dataset):
    return create_openai_request(
        series,
        vision=True,
        few_shots=train_dataset.few_shots(num_shots=1)
    )

In [ ]:
jsonl_fn = generate_batch_AD_requests(
    model_name='gpt-4o-mini',
    data_name='point',
    request_func=openai_request,
    variant="1shot_vision"
)

In [ ]:
from openai_api import openai_client
client = openai_client('gpt-4o-mini')

In [ ]:
batch_input_file = client.files.create(
    file=open(jsonl_fn, "rb"),
    purpose="batch"
)

In [ ]:
batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

In [ ]:
# client.batches.retrieve("batch_5OTeY7wGgmTGq1065wgoXsMT")
# client.batches.cancel("batch_L3JgGBmlv6yb5HRGH7GIs3mO")